# Classical CV Row Detection - Trimble Autonomous Solutions
#### Cody Weaver, Kaylee Engelhardt, Lara Chunko, Alex Mazur, David Crokett, Niharika Kunapuli, Ege Telatar

## Overview
We have designed an algorithm that will detect rows by using a combination of edge detection and and color detection. We use a sobel operator and we bitwise and the resulting edge image with the image produced by extracting green data from the image. The purpose of combining these methods is to reduce the amount of errors when detecting rows. **In the future will we be using difference of Gaussians (DoG) combined with sobel but we have run into issues with unexplainably long run times so we decided not to include DoG in this current version**

### Step 1  - Edge Detection: Sobel + DoG
Using the Sobel operator and DoG we have had good results at detecting plants. The top image is the result of running a Sobel operator on the image and the bottom image is the enhanced edge image using DoG. (This photo is from Kaylee's prensentaion "Classical Color Edge Detection " in the drive)

<br>
<img src='sobelEx.jpg' alt="Sobel" width=500>
<img src='DoGEx.jpg' alt='DoG' width=500>
<br>

### Step 2 - Color Detection: Color Masking
To detect rows using color we use apply a mask to the image. The result is an image that represents where the color was detected in the original image. This method can be adapted to any color. Below is an example of what applying the mask to an image will result in.

<br>
<img src='maskEx.jpg' alt='maskEx'>
<br>

### Step 3 - Row Detection
Once the image has been processed and the plants have been extracted from the image using step 1 and 2 we detect rows by splitting the image into horizontal sections. Once split into horizontal sections we detect rows turing the 3D image section into an signal representation of the rows. We scan the image section horizontally to determine the start and end points of a row.

<br>
<img src='sectionEx.jpg' alt='section'>

<img src='fig.jpg' alt='fig'>

In [1]:
####################################################################################
#
# CSCI 4308 CS Capstone
# Trimble Autonomous Solutions 
# Classical Computer Vision
#
####################################################################################

# imports 
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Green Masks
MASK_1 = np.asarray([36, 25, 25])
MASK_2 = np.asarray([86, 255, 255])

# DoG masks
DoG_MASK_1 = np.array([0, 0, 20])
DoG_MASK_2 = np.array([350,200,100])

""" IMAGE PROCESSING """

"""
	blurImage()
  input: image
  output: blurred image
"""
def blurImage(image, kernel, sigmaX, sigmaY):
    return cv2.GaussianBlur(image, kernel, sigmaX, sigmaY)

"""
	The SobelFunction() uses the sobel operator for edge detection
  inputs: image
  outputs: sobel image
"""
def sobelFunction(image):
    sobelx64f = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=3)
    abs_sobel64f = np.absolute(sobelx64f)
    sobel_8uy = np.uint8(abs_sobel64f)
    return sobel_8uy

"""
	maskImage()
  input: open cv image
  outputs: masked image 
"""
def maskImage(image, mask1, mask2):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    masked_image = cv2.inRange(hsv_image, mask1, mask2)
    return masked_image


"""
	Combine sobel image and masked image
  inputs: sobel image, color image
  outputs:
"""
def combineSobelAndColorMask(sobelImage, colorImage):
    combinedResult = cv2.bitwise_and(sobelImage, sobelImage, mask=colorImage)
    return combinedResult

"""
	Returns section of image 
  inputs: image, low row limit, high row limit, low col limit, high col limit
  output: section of image with corners (row1, col1) (row2, col2)
"""
def getImgSection(image, row1, row2, col1, col2):
    rows, cols, _ = np.shape(image)
    if row1 < 0 or row2 >= rows: return None
    if col1 < 0 or col2 >= cols: return None
  
    return image[row1:row2, col1:col2]

""" HELPER FUNCTIONS FOR DETECTING ROWS """

"""
	Computes green signal from image section
  inputs: image section, signal threshold
  output: green signal stored as a 1D array
"""
def getGreenSignal(section, threshold):
    rows, cols, _ = np.shape(section)
    greenSignal = []

    for i in range(cols):
        greenSum = 0
        for j in range(rows):
            greenSum += sum(section[j][i])

        if greenSum >= threshold:
            greenSignal.append(1)
        else:
            greenSignal.append(0)

    return greenSignal

"""
	Gets rows from green signal and returns list of row starting and stopping positions
	inputs: green signal array, and max gap allowed between rows
"""
def getRows(greenSignal, maxRowGap):
    rows = []
    row = -1
    for i in range(len(greenSignal)):
        if greenSignal[i] == 1:
            if row == -1:
                if rows != [] and i - rows[-1][1] <= maxRowGap:
                    row = rows[-1][0]
                    rows.pop()
                else:
                    row = i
        else:
            if row != -1:
                rows.append((row, i))
                row = -1
            
    return rows


"""
 Returns the left, center, and right rows detected for each section
 Inputs: rows - list of rows detected in section, prevCenter - horizontal position of prev center row
 Returns: (filteredRows, centerCoord)
"""
def filterRows(rows, prevCenter):
    midIndex = -1
    minDist = np.inf
    for i in range(len(rows)): # finds row detected that is closest to prevCenter
        if rows[i][0] > 620 and rows[i][0] < 820:
            rowCenter = (rows[i][0] + rows[i][1])//2
            if np.abs(prevCenter - rowCenter) < minDist:
                minDist = np.abs(prevCenter - rowCenter)
                midIndex = i
    
    if midIndex > 0 and midIndex < len(rows) - 1: # if at there is at least row detected on each side of the center row
        filteredRows = [rows[i] for i in range(midIndex-1, midIndex+2)]
    else: # don't return rows unless at least 3 are detected
        filteredRows = []
        
    if filteredRows != []: # calc new center point
        center = (rows[midIndex][0] + rows[midIndex][1])//2
    else: # doesn't update center if rows not detected
        center = prevCenter
    
    return filteredRows, center
       
"""
    Takes a track, as a array of tuples (x,y) and smooths the track by 
    averaging the horizontal poisition of the previous, current, and next center point
    Inputs: track - arr of tuples
    Return: smoothed Track as an array of tuples
"""
def smoothTrack(track):
    newTrack = []
        
    newTrack.append(track[0])
    
    for i in range(1, len(track) - 1):
            newPoint = ((track[i-1][0] + track[i][0] + track[i+1][0])//3, track[i][1])
            newTrack.append(newPoint)
    
    newTrack.append(track[len(track) - 1])
    
    return newTrack    

def computeFrame(image):
    rows, cols, _ = np.shape(image)
    min_col = 240
    max_col = 1680
    threshold = 1000
    prevCenter = 720
    blurredImage = blurImage(image, kernel=(7,7), sigmaX=0, sigmaY=0)

    # compute edge detection
    sobelImage = sobelFunction(blurredImage)

    # isolate color
    maskedImage = maskImage(image, MASK_1, MASK_2)

    # combine the edge detection results and isolated color results into one image
    combinedImage = combineSobelAndColorMask(sobelImage, maskedImage)
    
    leftTrack = []
    rightTrack = []
    
    leftRows = []
    centerRows = []
    rightRows = []
    
    for section_end in range(1050, 150, -50):
        section = getImgSection(combinedImage, section_end-50, section_end, min_col, max_col)
        
        greenSignal = getGreenSignal(section,threshold)
        if section_end == 400:
            plt.plot(greenSignal)
        
        if section_end > 400:
            maxRowGap = 100
        else:
            maxRowGap = 50
        
        rows = getRows(greenSignal, maxRowGap)
        if len(rows) >= 3:
            filteredRows, prevCenter = filterRows(rows, prevCenter)
        else:
            filteredRows = []
            
        for row in filteredRows: # draw row boxes on image
            if row != None:
                cv2.rectangle(image, (row[0]+min_col, section_end-50), (row[1]+min_col, section_end), (255, 0, 0), thickness=2)
        
        if filteredRows != []: # caclulates center points for left and right tracdks
            leftTrack.append(((filteredRows[1][0] + filteredRows[0][1])//2 + min_col, section_end-25))
            rightTrack.append(((filteredRows[2][0] + filteredRows[1][1])//2 + min_col, section_end-25))
            
            leftRows.append((filteredRows[0], section_end))
            centerRows.append((filteredRows[1], section_end))
            rightRows.append((filteredRows[2], section_end))
            
    leftTrack = smoothTrack(leftTrack)
    rightTrack = smoothTrack(rightTrack)
    
    rowWidthSum = 0
    for i in range(0, len(leftRows)):
        rowWidthSum += leftRows[i][0][1] - leftRows[i][0][0]
        
    avgRowWidth = rowWidthSum//len(leftRows)
    
    # draw row lines on image
    for i in range(len(leftTrack) - 1):
        cv2.line(image, leftTrack[i], leftTrack[i+1], (0, 0, 255), thickness=2)
    
    for i in range(len(rightTrack) - 1):
        cv2.line(image, rightTrack[i], rightTrack[i+1], (0, 0, 255), thickness=2)
        
    cv2.line(image, (960, 0), (960, 1080), (255,255,255), thickness=4)
        
    return image

"""
	runCodeOnVideo()
  inputs: video, iterNum
  outputs:
"""
def runCodeOnVideo(video, iterNum):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('vid/output4.MP4', fourcc, 20.0, (1920,1080))
  
    # Check if camera opened successfully
    if (video.isOpened()== False):
          print("Error opening video stream or file")

    # make an iterator so we only look at a couple frames rather than the whole video
    iteration = 0

    # Read until video is completed
    while(video.isOpened() and iteration < iterNum):
    # Capture frame-by-frame
        ret, frame = video.read()
        if ret == True:
            rowImage = computeFrame(frame)
            
            plt.imshow(rowImage)
            plt.show()
            out.write(rowImage)

        # Break the loop
        else:
            pass

        iteration +=1

    # When everything done, release the video capture object
    video.release()
    out.release()

In [3]:
#image = cv2.imread('img/test_img.jpg')
#cap = cv2.VideoCapture('vid/Corn_Row_Field_Sun_n_Shade.MP4')
#runCodeOnVideo(cap, 10400)